## Índice

1. [General Statistics](#1)
2. [Boxplots](#2)
3. [First Integer Helicity Betta vs Gamma](#3)
4. [Sum Absolute Helicity Betta vs Gamma](#4)
5. [First Integer Enstrophy Betta vs Gamma](#5)
6. [3D Curl Comparisons](#6)
7. [Individual 3D Curl Baseline vs Experimental Color](#7)
8. [Velocity Field Baseline Nivel](#8)
9. [Velocity Field MEI Nivel](#9)
10. [Comparison Moduli before and after for MEI](#10)
11. [Comparison Derivatives Histograms](#11)
12. [Comparison Time Series Laser](#12)


In [2]:
root_folder = r"F:\Datos\12Ago24"

# Horarios ajustados de mediciones de línea base y de intervenciones experimentales

lab_b_matutino_base_times = {
    '01': "10:39:59.065",
    '02': "11:19:56.844",
    '03': "12:00:00.896",
    '04': "12:39:59.139",
    '05': "13:19:59.461"
}

# Horarios de intervenciones experimentales
lab_b_matutino_intervention_times = {
    '01': "10:59:59.414",
    '02': "11:39:59.104",
    '03': "12:19:59.778",
    '04': "13:00:01.295"
}

# Horarios de mediciones de línea base para Lab_A en turno matutino
lab_a_matutino_base_times = {
    '01': "10:39:59.065",
    '02': "11:19:56.844",
    '03': "12:00:00.896",
    '04': "12:39:59.139",
    '05': "13:19:59.461"
}

# Horarios de intervenciones experimentales para Lab_A en turno matutino
lab_a_matutino_intervention_times = {
    '01': "10:59:59.414",
    '02': "11:39:59.104",
    '03': "12:19:59.778",
    '04': "13:00:01.295"
}

n_change = {"Lab_Betta":"Lab_Betta", "Lab_Gamma":"Lab_Gamma"}

triad_names = {
    "FRT": "Frontal-Derecho-Superior",  
    "PLB": "Trasero-Izquierdo-Inferior",  
    "FLT": "Frontal-Izquierdo-Superior",  
    "PRB": "Trasero-Derecho-Inferior",  
    "FRB": "Frontal-Derecho-Inferior",  
    "PLT": "Trasero-Izquierdo-Superior",  
    "FLB": "Frontal-Izquierdo-Inferior",  
    "PRT": "Trasero-Derecho-Superior",  
    "RTB": "Derecho-Superior-Inferior",  
    "FLP": "Frontal-Izquierdo-Trasero",  
    "LTB": "Izquierdo-Superior-Inferior",  
    "FRP": "Frontal-Derecho-Trasero"
}

## 1

$$
\Huge \text{General Statistcs}
$$

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew, iqr
from matplotlib.backends.backend_pdf import PdfPages
import openpyxl
from datetime import datetime

# Función para leer datos del sensor desde un archivo
def read_sensor_data(filepath):
    data = []
    timestamps = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return timestamps, np.array(data).T

# Función para analizar el contenido de una carpeta
def analyze_folder(root_folder):
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        # Evitar carpetas llamadas "Data Analysis"
        if 'Data Analysis' in dirs:
            dirs.remove('Data Analysis')

        for filename in files:
            if filename.startswith("0") and filename.endswith(("negra.txt")):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = (timestamps, data, filename)
    return data_per_file

# Función para generar estadísticas de los datos
def generate_statistics(data):
    return {
        'mean': np.mean(data, axis=1),
        'median': np.median(data, axis=1),
        'variance': np.var(data, axis=1),
        'stdDev': np.std(data, axis=1),
        'min': np.min(data, axis=1),
        'max': np.max(data, axis=1),
        'firstQuartile': np.percentile(data, 25, axis=1),
        'thirdQuartile': np.percentile(data, 75, axis=1),
        'iqr': iqr(data, axis=1),
        'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data],
        'kurtosis': kurtosis(data, axis=1, fisher=True, bias=False),
        'skewness': skew(data, axis=1, bias=False)
    }

# Función para extraer la fecha del directorio root_folder
def extract_date_from_root_folder(root_folder):
    return os.path.basename(root_folder)

# Función para formatear las estadísticas
def format_statistics(stats, fmt=".10f"):
    return {key: [f"{val:{fmt}}" for val in values] for key, values in stats.items()}

# Función para asignar colores basados en el sufijo del archivo
def get_color_for_file(filename):
    if "negra" in filename:
        return "black"


# Función para generar los gráficos y los PDFs
def plot_comparison_grid(data_per_file, output_pdf_folder, output_img_folder, output_text_folder, lab, session, date, workbook):
    component_names = ["FRONT(A0)", "REAR(A1)", "LEFT(A2)", "RIGHT(A3)", "TOP(A4)", "BOTTOM(A5)"]
    session_name = "Vespertino" if session == "V" else "Matutino"
    sheet = workbook.active

    for filename, (timestamps, data, file_path) in data_per_file.items():
        color = get_color_for_file(filename)  # Color basado en el sufijo del archivo

        # Determinar las subcarpetas Baseline o Experimental Color
        if "mednegra.txt" in filename:
            subfolder = "Baseline"
        else:
            subfolder = "Experimental Color"

        pdf_file = os.path.join(output_pdf_folder, subfolder, f"{filename}.pdf")
        img_file = os.path.join(output_img_folder, subfolder, f"{filename}.png")

        with PdfPages(pdf_file) as pdf:
            if lab in n_change:
                n_lab = n_change[lab]
            fig, axes = plt.subplots(6, 3, figsize=(36, 45))  # Tamaño más grande de la figura
            fig.subplots_adjust(top=0.85, wspace=0.6)  # Más espacio entre columnas, se puede modificar wspace para ajustar el espacio
            fig.suptitle(f"Análisis del archivo: {filename}\n"
                         f"Laboratorio: {n_lab}, Turno: {session_name}, Fecha: {date}", fontsize=40, y=0.98)  # Título más grande
            axes = axes.flatten()
            stats = generate_statistics(data)  # Generar estadísticas una vez por archivo
            stats_formatted = format_statistics(stats, fmt=".10f")

            for i in range(6):
                # Gráfico de Datos Crudos
                ax = axes[3 * i]
                ax.plot(data[i], label='Datos', color=color)
                ax.set_title(f"Componente {component_names[i]} - Datos Crudos", fontsize=27)  # Títulos más grandes
                ax.set_xlabel("Registros", fontsize=25)
                ax.set_ylabel("Volts", fontsize=25)
                ax.legend()
                ax.tick_params(axis='x', rotation=45, labelsize=25)  # Tamaño de las etiquetas de los ejes x y y
                ax.tick_params(axis='y', labelsize=25)  # Tamaño de las etiquetas de los ejes y

                # Histograma de Derivadas
                derivative = np.gradient(data[i])
                ax_hist = axes[3 * i + 1]
                ax_hist.hist(derivative, bins=25, color=color, edgecolor='black')
                ax_hist.set_title(f"Componente {component_names[i]} - Histograma de Derivadas", fontsize=27)
                ax_hist.set_xlabel("Valor de la Derivada", fontsize=25)
                ax_hist.set_ylabel("Frecuencia", fontsize=25)
                ax_hist.tick_params(axis='x', labelsize=25)  # Tamaño de las etiquetas de los ejes x
                ax_hist.tick_params(axis='y', labelsize=25)  # Tamaño de las etiquetas de los ejes y

                # Texto de Estadísticas de Datos Crudos
                stats_text = (f"Media: {stats_formatted['mean'][i]}\n"
                              f"Mediana: {stats_formatted['median'][i]}\n"
                              f"Varianza: {stats_formatted['variance'][i]}\n"
                              f"Desviación Estándar: {stats_formatted['stdDev'][i]}\n"
                              f"Mínimo: {stats_formatted['min'][i]}\n"
                              f"Máximo: {stats_formatted['max'][i]}\n"
                              f"Primer Cuartil: {stats_formatted['firstQuartile'][i]}\n"
                              f"Tercer Cuartil: {stats_formatted['thirdQuartile'][i]}\n"
                              f"IQR: {stats_formatted['iqr'][i]}\n"
                              f"Varianza Recortada: {stats_formatted['trimmedVariance'][i]}\n"
                              f"Kurtosis: {stats_formatted['kurtosis'][i]}\n"
                              f"Asimetría: {stats_formatted['skewness'][i]}")
                ax_stats = axes[3 * i + 2]
                ax_stats.text(0.1, 0.5, stats_text, fontsize=25, verticalalignment='center')
                ax_stats.axis('off')
                ax_stats.set_title(f"Componente {component_names[i]} - Estadísticas de Datos Crudos", fontsize=24)

            plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta el espacio de toda la figura
            pdf.savefig(fig)  # Guarda el gráfico en el PDF
            plt.savefig(img_file, dpi=400)  # Guarda el gráfico como imagen de alta calidad
            plt.close()

            # Añadir datos al Excel
            add_data_to_excel(sheet, stats_formatted, date, session_name, filename, lab)

        # Generar archivo de texto con las estadísticas
        text_folder = os.path.join(output_text_folder, subfolder)
        stats_file = os.path.join(text_folder, f"{filename}.txt")
        os.makedirs(os.path.dirname(stats_file), exist_ok=True)
        with open(stats_file, 'w', encoding='utf-8') as f:
            f.write(f"{date},{session_name},{filename}\n\n")
            for i, component in enumerate(component_names):
                f.write(f"{component}\n")
                f.write(f"{stats_formatted['median'][i]},{stats_formatted['mean'][i]},{stats_formatted['variance'][i]},{stats_formatted['stdDev'][i]},{stats_formatted['min'][i]},{stats_formatted['max'][i]},{stats_formatted['firstQuartile'][i]},{stats_formatted['thirdQuartile'][i]},{stats_formatted['iqr'][i]},{stats_formatted['trimmedVariance'][i]},{stats_formatted['kurtosis'][i]},{stats_formatted['skewness'][i]}\n\n")

# Función para añadir datos al Excel
def add_data_to_excel(sheet, stats_formatted, date, session_name, filename, lab):
    component_names = ["FRONT(A0)", "REAR(A1)", "LEFT(A2)", "RIGHT(A3)", "TOP(A4)", "BOTTOM(A5)"]
    row_data = [date, session_name, lab, filename]  # Incluyendo el nombre del laboratorio

    for i in range(6):
        row_data.extend([
            stats_formatted['median'][i], stats_formatted['mean'][i], stats_formatted['variance'][i],
            stats_formatted['stdDev'][i], stats_formatted['min'][i], stats_formatted['max'][i],
            stats_formatted['firstQuartile'][i], stats_formatted['thirdQuartile'][i], stats_formatted['iqr'][i],
            stats_formatted['trimmedVariance'][i], stats_formatted['kurtosis'][i], stats_formatted['skewness'][i]
        ])
        row_data.append("")  # Espacio entre componentes

    sheet.append(row_data)

# Función para procesar todos los experimentos
def process_all_experiments(root_folder):
    date = extract_date_from_root_folder(root_folder)
    date = date.replace("- copia", "").strip()
    labs = ["Lab_Betta", "Lab_Gamma"]
    sessions = ["V", "M"]

    # Creando un nuevo libro de trabajo y hoja
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Datos"

    # Añadiendo la fila de títulos
    titles = [
        "DATE", "SHIFT", "LAB", "FILENAME",
        "General Statistics FRONT", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics REAR", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics LEFT", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics RIGHT", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics TOP", "", "", "", "", "", "", "", "", "", "", "", "",
        "General Statistics BOTTOM", "", "", "", "", "", "", "", "", "", "", "", ""
    ]
    sheet.append(titles)

    # Añadiendo subtítulos
    subtitles = [
        "", "", "", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", "",
        "Median", "Mean", "Variance", "STD", "Min", "Max", "First Quartile", "Third Quartile", "IQR", "Trimmed Variance (0.2)", "Kurtosis", "Skewness", ""
    ]
    sheet.append(subtitles)

    for lab in labs:
        for session in sessions:
            experiment_folder = os.path.join(root_folder, f"{date} - {lab}", f"{date}.{session} - {lab}")
            print(f"Checking directory: {experiment_folder}")
            if os.path.exists(experiment_folder):
                output_pdf_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "General Statistics", "PDFs")
                output_img_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "General Statistics", "Images")
                output_text_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "General Statistics Text")
                os.makedirs(output_pdf_folder, exist_ok=True)
                os.makedirs(output_img_folder, exist_ok=True)
                os.makedirs(output_text_folder, exist_ok=True)

                # Crear subcarpetas Baseline y Experimental Color
                os.makedirs(os.path.join(output_pdf_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_pdf_folder, "Experimental Color"), exist_ok=True)
                os.makedirs(os.path.join(output_img_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_img_folder, "Experimental Color"), exist_ok=True)
                os.makedirs(os.path.join(output_text_folder, "Baseline"), exist_ok=True)
                os.makedirs(os.path.join(output_text_folder, "Experimental Color"), exist_ok=True)

                data_per_file = analyze_folder(experiment_folder)
                plot_comparison_grid(data_per_file, output_pdf_folder, output_img_folder, output_text_folder, lab, session, date, workbook)
            else:
                print(f"Directory does not exist: {experiment_folder}")

    # Ruta de salida del archivo Excel
    excel_output_path = os.path.join(root_folder, f"General_Statistics_{date}.xlsx")

    # Asegurarse de que el directorio existe antes de guardar
    os.makedirs(root_folder, exist_ok=True)

    # Guardando el archivo Excel al final de todos los experimentos
    workbook.save(excel_output_path)
    print("Archivo Excel generado exitosamente en:", excel_output_path)

# Cambia la ruta de root_folder a la que corresponde a tu directorio raíz

process_all_experiments(root_folder)

Checking directory: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.V - Lab_Betta
Directory does not exist: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.V - Lab_Betta
Checking directory: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.M - Lab_Betta
Checking directory: E:\Pruebas\14Sep24\14Sep24 - Lab_Gamma\14Sep24.V - Lab_Gamma
Directory does not exist: E:\Pruebas\14Sep24\14Sep24 - Lab_Gamma\14Sep24.V - Lab_Gamma
Checking directory: E:\Pruebas\14Sep24\14Sep24 - Lab_Gamma\14Sep24.M - Lab_Gamma
Archivo Excel generado exitosamente en: E:\Pruebas\14Sep24\General_Statistics_14Sep24.xlsx


## 2

$$
\Huge \text{Boxplots}
$$

In [3]:
#Boxplots
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import iqr
import re

def read_sensor_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data).T
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return None, None

def calibrate_sensor(data, filename):
    try:
        results = {
            'mean': np.mean(data, axis=1),
            'median': np.median(data, axis=1),
            'variance': np.var(data, axis=1),
            'stdDev': np.std(data, axis=1),
            'min': np.min(data, axis=1),
            'max': np.max(data, axis=1),
            'firstQuartile': np.percentile(data, 25, axis=1),
            'thirdQuartile': np.percentile(data, 75, axis=1),
            'iqr': iqr(data, axis=1),
            'trimmedVariance': [np.var(np.trim_zeros(np.sort(sensor_data))) for sensor_data in data]
        }
        return results
    except Exception as e:
        print(f"Error al cacular estadisticas los datos, REVISAR ARCHIVO: {filename}: {e}")
        return None


def find_experiment_folder(root_folder, lab, session):
    # Expresión regular para buscar carpetas con formato de fecha
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    msg_error = 'no se hayó la carpeta'
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and session in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return msg_error



# Función para generar boxplots
def generate_boxplot(data, filename, output_folder, lab, session):
    labels = ["Frontal", "Rear", "Left", "Right", "Top", "Bottom"]
    color_dict = {
        'negra': 'black',
        'roja': 'red',
        'morada': 'purple',
        'azul': 'blue',
        'verde': 'green',
        'amarilla': 'yellow'
    }
    file_base_name = filename.split('.')[0]  # Remove extension
    color_suffix = file_base_name.split('med')[-1]
    color = color_dict.get(color_suffix, 'gray')  # Default color if not matched
    if lab in n_change:
      n_lab = n_change[lab]
    plt.figure(figsize=(10, 6))
    plt.boxplot(data.T, notch=True, patch_artist=True, boxprops=dict(facecolor=color))
    plt.title(f'Box Plot for {filename} - {n_lab} - {session}')
    plt.xlabel('Side of Cube')
    plt.ylabel('Volts')
    plt.xticks(range(1, len(labels) + 1), labels, rotation=45)
    plt.savefig(os.path.join(output_folder, f"{file_base_name}_{lab}_{session}_boxplot.png"))
    plt.close()

# Función para generar boxplots de comparación
def generate_comparison_boxplot(data_a, data_b, filename, output_folder, session, color_a, color_b):
    labels = ["Frontal", "Rear", "Left", "Right", "Top", "Bottom"]
    combined_data = [data_a[i] for i in range(data_a.shape[0])] + [data_b[i] for i in range(data_b.shape[0])]
    positions = list(range(1, data_a.shape[0] + 1)) + list(range(data_a.shape[0] + 2, data_a.shape[0] + data_b.shape[0] + 2))
    colors = [color_a] * data_a.shape[0] + [color_b] * data_b.shape[0]

    plt.figure(figsize=(12, 8))
    boxprops = dict(facecolor='white')  # Default color
    bplots = plt.boxplot(combined_data, positions=positions, notch=True, patch_artist=True, boxprops=boxprops)

    for patch, color in zip(bplots['boxes'], colors):
        patch.set_facecolor(color)

    plt.title(f'Comparison Box Plot for {filename} - {session}\nLab_Betta (left) vs Lab_Gamma (right)')
    plt.xlabel('Side of Cube')
    plt.ylabel('Volts')
    plt.xticks(range(1, len(labels) * 2 + 1), labels * 2, rotation=45)
    plt.legend(handles=[plt.Line2D([0], [0], color=color_a, lw=4, label='Lab_Betta'),
                        plt.Line2D([0], [0], color=color_b, lw=4, label='Lab_Gamma')],
               labels=['Lab_Betta', 'Lab_Gamma'], loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=2)
    plt.savefig(os.path.join(output_folder, f"{filename}_comparison_{session}_boxplot.png"))
    plt.close()

# Función para analizar las carpetas
def analyze_folder(folder_path, output_folder, lab, session):
    file_stats = {}
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if any(filename.startswith("0") and filename.endswith(ext) for ext in ["mednegra.txt", "medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt"]):
                file_path = os.path.join(root, filename)
                timestamps, data = read_sensor_data(file_path)
                if data is not None:
                    stats = calibrate_sensor(data, filename)
                    if stats is not None:
                        file_stats[filename] = {'stats': stats, 'data': data}
                        generate_boxplot(data, filename, output_folder, lab, session)
    return file_stats


# Función para procesar todos los experimentos
def process_all_experiments(root_folder):
    labs = ["Lab_Betta", "Lab_Gamma"]
    sessions = ["V", "M"]
    all_data = {session: {lab: {} for lab in labs} for session in sessions}
    global_output_folder = os.path.join(root_folder, "Data Analysis", "Global Comparisons")
    os.makedirs(global_output_folder, exist_ok=True)

    for lab in labs:
        for session in sessions:
            experiment_folder = find_experiment_folder(root_folder, lab, session)
            if os.path.exists(experiment_folder):
                output_folder_base = os.path.join(experiment_folder, "Data Analysis", "Graphics")
                function_title = "Boxplots"
                output_folder = os.path.join(output_folder_base, function_title)
                os.makedirs(output_folder, exist_ok=True)
                file_stats = analyze_folder(experiment_folder, output_folder, lab, session)
                all_data[session][lab] = file_stats

    color_dict = {
        'negra': 'black',
        'roja': 'red',
        'morada': 'purple',
        'azul': 'blue',
        'verde': 'green',
        'amarilla': 'yellow'
    }

    # Generar comparaciones globales
    for session in sessions:
        for filename in all_data[session]["Lab_Betta"]:
            if filename in all_data[session]["Lab_Gamma"]:
                data_a = all_data[session]["Lab_Betta"][filename]['data']
                data_b = all_data[session]["Lab_Gamma"][filename]['data']
                color_suffix_a = filename.split('med')[-1].split('.')[0]
                color_suffix_b = filename.split('med')[-1].split('.')[0]
                color_a = color_dict.get(color_suffix_a, 'blue')
                color_b = color_dict.get(color_suffix_b, 'green')
                generate_comparison_boxplot(data_a, data_b, filename, global_output_folder, session, color_a, color_b)

# Procesar todos los experimentos

process_all_experiments(root_folder)


## 3
$$
\Huge \text{First Integer Helicity Betta vs Gamma}
$$

In [4]:
#Grafica Integral Helicidad Betta vs Gamma
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob
import re

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)
                values.append(float(parts[1]))
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

# Diccionario de nombres de triadas
triad_names = {
    "FRT": "Frontal-Derecho-Superior",  
    "PLB": "Trasero-Izquierdo-Inferior",  
    "FLT": "Frontal-Izquierdo-Superior",  
    "PRB": "Trasero-Derecho-Inferior",  
    "FRB": "Frontal-Derecho-Inferior",  
    "PLT": "Trasero-Izquierdo-Superior",  
    "FLB": "Frontal-Izquierdo-Inferior",  
    "PRT": "Trasero-Derecho-Superior",  
    "RTB": "Derecho-Superior-Inferior",  
    "FLP": "Frontal-Izquierdo-Trasero",  
    "LTB": "Izquierdo-Superior-Inferior",  
    "FRP": "Frontal-Derecho-Trasero"
}

def plot_comparison(base_file1_betta, base_file2_betta, experimental_file_betta,
                    base_file1_gamma, base_file2_gamma, experimental_file_gamma,
                    output_dir, shift, date_str):
    # Leer los datos de Lab_Betta
    timestamps_base1_betta, base_values1_betta = read_data(base_file1_betta)
    timestamps_base2_betta, base_values2_betta = read_data(base_file2_betta)
    timestamps_experimental_betta, experimental_values_betta = read_data(experimental_file_betta)

    # Leer los datos de Lab_Gamma
    timestamps_base1_gamma, base_values1_gamma = read_data(base_file1_gamma)
    timestamps_base2_gamma, base_values2_gamma = read_data(base_file2_gamma)
    timestamps_experimental_gamma, experimental_values_gamma = read_data(experimental_file_gamma)

    # Reiniciar todas las estampas de tiempo para Lab_Betta
    start_time_base1_betta = timestamps_base1_betta[0]
    start_time_base2_betta = timestamps_base2_betta[0]
    start_time_experimental_betta = timestamps_experimental_betta[0]
    timestamps_base1_betta = [(ts - start_time_base1_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_base1_betta]
    timestamps_base2_betta = [(ts - start_time_base2_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_base2_betta]
    timestamps_experimental_betta = [(ts - start_time_experimental_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental_betta]

    # Reiniciar todas las estampas de tiempo para Lab_Gamma
    start_time_base1_gamma = timestamps_base1_gamma[0]
    start_time_base2_gamma = timestamps_base2_gamma[0]
    start_time_experimental_gamma = timestamps_experimental_gamma[0]
    timestamps_base1_gamma = [(ts - start_time_base1_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_base1_gamma]
    timestamps_base2_gamma = [(ts - start_time_base2_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_base2_gamma]
    timestamps_experimental_gamma = [(ts - start_time_experimental_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental_gamma]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    base_times1_betta = mdates.date2num(timestamps_base1_betta)
    base_times2_betta = mdates.date2num(timestamps_base2_betta)
    experimental_times_betta = mdates.date2num(timestamps_experimental_betta)

    base_times1_gamma = mdates.date2num(timestamps_base1_gamma)
    base_times2_gamma = mdates.date2num(timestamps_base2_gamma)
    experimental_times_gamma = mdates.date2num(timestamps_experimental_gamma)
    
    # Obtener las triadas para los títulos
    triada_betta = extract_triada(experimental_file_betta)
    triada_gamma = extract_triada(experimental_file_gamma)

    # Usar los nombres descriptivos de las triadas en los títulos
    triad_name_betta = triad_names.get(triada_betta, triada_betta)
    triad_name_gamma = triad_names.get(triada_gamma, triada_gamma)

    # Lab_Betta gráfico
    ax1.plot(base_times1_betta, base_values1_betta, c='black', label=os.path.basename(base_file1_betta))
    ax1.plot(base_times2_betta, base_values2_betta, c='gray', label=os.path.basename(base_file2_betta))
    ax1.plot(experimental_times_betta, experimental_values_betta, c=get_color(experimental_file_betta), label=os.path.basename(experimental_file_betta))
    ax1.set_title(f'Lab_Betta - {shift} - Date: {date_str} - {triad_name_betta}', fontsize=14)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Value')
    ax1.legend()
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    ax1.grid(True)

    # Lab_Gamma gráfico
    ax2.plot(base_times1_gamma, base_values1_gamma, c='black', label=os.path.basename(base_file1_gamma))
    ax2.plot(base_times2_gamma, base_values2_gamma, c='gray', label=os.path.basename(base_file2_gamma))
    ax2.plot(experimental_times_gamma, experimental_values_gamma, c=get_color(experimental_file_gamma), label=os.path.basename(experimental_file_gamma))
    ax2.set_title(f'Lab_Gamma - {shift} - Date: {date_str} - {triad_name_gamma}', fontsize=14)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Value')
    ax2.legend()
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    ax2.grid(True)

    plt.tight_layout()
    
    # Generar un nombre más corto para cada gráfico
    number1_betta = extract_number(base_file1_betta)
    number2_betta = extract_number(base_file2_betta)
    number_exp_betta = extract_number(experimental_file_betta)
    number1_gamma = extract_number(base_file1_gamma)
    number2_gamma = extract_number(base_file2_gamma)
    number_exp_gamma = extract_number(experimental_file_gamma)

    output_filename = (f"Helicity_Comparison_{number1_betta}_{number2_betta}_{number_exp_betta}_"
                   f"{number1_gamma}_{number2_gamma}_{number_exp_gamma}.png")

    # Determinar la carpeta de salida en función del código de triada del archivo experimental de Lab_Betta
    triad_code_betta = re.search(r'helicity_(\w+)_', experimental_file_betta)
    if triad_code_betta:
        triad_code_betta = triad_code_betta.group(1)
        specific_output_dir = os.path.join(output_dir, triad_code_betta)
    else:
        specific_output_dir = output_dir

    if not os.path.exists(specific_output_dir):
        os.makedirs(specific_output_dir)

    output_filepath = os.path.join(specific_output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    # Extraer triadas de los nombres de los archivos
    baseline_triads = set()
    experimental_triads = set()

    for f in baseline_files:
        match = re.search(r'helicity_(\w+)_', f)
        if match:
            baseline_triads.add(match.group(1))

    for f in experimental_files:
        match = re.search(r'helicity_(\w+)_', f)
        if match:
            experimental_triads.add(match.group(1))

    # Generar combinaciones solo para triadas comunes
    common_triads = baseline_triads.intersection(experimental_triads)
    for triad in common_triads:
        baseline_files_for_triad = [f for f in baseline_files if f"helicity_{triad}_" in f]
        experimental_files_for_triad = [f for f in experimental_files if f"helicity_{triad}_" in f]
        
        # Asegurarse de que hay al menos dos archivos base para cada triada
        if len(baseline_files_for_triad) >= 2:
            for i, base_file1 in enumerate(baseline_files_for_triad):
                for base_file2 in baseline_files_for_triad[i+1:]:
                    for exp_file in experimental_files_for_triad:
                        number1 = int(extract_number(base_file1))
                        number2 = int(extract_number(base_file2))
                        number_exp = int(extract_number(exp_file))
                        if number1 == number_exp == ((number2)-1):
                            combinations.append((base_file1, base_file2, exp_file))
    return combinations

def extract_triada(ruta):
    # Extrae el nombre del archivo de la ruta
    nombre_archivo = os.path.basename(ruta)
    
    # Busca y extrae la triada
    match_triada = re.search(r'(FLB|FRP|PLB|PRB|FRT|PRT|FLT|PLT|RTB|LTB|FRB|FLP)', nombre_archivo)
    triada = match_triada.group(1) if match_triada else None
    
    return triada

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    combinations1 = []  # Definir una lista para combinaciones de Lab_Betta
    combinations2 = []  # Definir una lista para combinaciones de Lab_Gamma

    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Helicity_Experimental_Color")
            output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Helicity_Comp")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Buscar en subcarpetas de triadas
            baseline_files = []
            experimental_files = []

            for triad_code in triad_codes.values():
                # Buscar en subcarpetas basadas en triad_codes
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'sum_helicity_*mednegra.txt')))
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'sum_helicity_*med*.txt')))

            if lab == "Lab_Betta":
                combinations1 = generate_combinations(baseline_files, experimental_files)
                combinations1 = list(set(combinations1))
            else:
                combinations2 = generate_combinations(baseline_files, experimental_files)
                combinations2 = list(set(combinations2))

    for base_file1_betta, base_file2_betta, experimental_file_betta in combinations1:
        for base_file1_gamma, base_file2_gamma, experimental_file_gamma in combinations2:
            number1_betta = extract_number(base_file1_betta)
            number2_betta = extract_number(base_file2_betta)
            number_exp_betta = extract_number(experimental_file_betta)
            number1_gamma = extract_number(base_file1_gamma)
            number2_gamma = extract_number(base_file2_gamma)
            number_exp_gamma = extract_number(experimental_file_gamma)
            triada1_betta = extract_triada(base_file1_betta)
            triada2_betta = extract_triada(base_file2_betta)
            triada_exp_betta = extract_triada(experimental_file_betta)
            triada1_gamma = extract_triada(base_file1_gamma)
            triada2_gamma = extract_triada(base_file1_gamma)
            triada_exp_gamma = extract_triada(experimental_file_gamma)
            
            if (number1_betta == number1_gamma and number2_betta == number2_gamma and number_exp_betta == number_exp_gamma and 
                triada1_betta == triada2_betta == triada_exp_betta and 
                triada1_gamma == triada2_gamma == triada_exp_gamma and 
                triada1_betta == triada1_gamma and 
                triada2_betta == triada2_gamma and 
                triada_exp_betta == triada_exp_gamma):
                plot_comparison(base_file1_betta, base_file2_betta, experimental_file_betta,
                            base_file1_gamma, base_file2_gamma, experimental_file_gamma,
                            output_directory, shift, date_str)

if __name__ == "__main__":
    main()


Omitiendo cálculo para Lab_Betta V ya que el directorio no existe.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\FLT_velocity\Helicity_Comparison_02_03_02_02_03_02.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\FRT_velocity\Helicity_Comparison_04_05_04_04_05_04.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\RTB_velocity\Helicity_Comparison_01_02_01_01_02_01.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\LTB_velocity\Helicity_Comparison_04_05_04_04_05_04.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\PLB_velocity\Helicity_Comparison_02_03_02_02_03_02.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\FLB_velocity\Helicity_Comparison_02_03_02_02_03_02.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Helicity_Comp\PLT_velocity\Helic




## 4 
$$
\Huge \text{Sum Absolute Helicity Betta vs Gamma}
$$


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob
import re

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)
                values.append(float(parts[1]))
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

# Diccionario de triadas con nombres descriptivos
triad_names = {
    "FRT": "Frontal-Derecho-Superior",  
    "PLB": "Trasero-Izquierdo-Inferior",  
    "FLT": "Frontal-Izquierdo-Superior",  
    "PRB": "Trasero-Derecho-Inferior",  
    "FRB": "Frontal-Derecho-Inferior",  
    "PLT": "Trasero-Izquierdo-Superior",  
    "FLB": "Frontal-Izquierdo-Inferior",  
    "PRT": "Trasero-Derecho-Superior",  
    "RTB": "Derecho-Superior-Inferior",  
    "FLP": "Frontal-Izquierdo-Trasero",  
    "LTB": "Izquierdo-Superior-Inferior",  
    "FRP": "Frontal-Derecho-Trasero"
}

def plot_comparison(base_file1_betta, base_file2_betta, experimental_file_betta,
                    base_file1_gamma, base_file2_gamma, experimental_file_gamma,
                    output_dir, shift, date_str):
    # Leer los datos de Lab_Betta
    timestamps_base1_betta, base_values1_betta = read_data(base_file1_betta)
    timestamps_base2_betta, base_values2_betta = read_data(base_file2_betta)
    timestamps_experimental_betta, experimental_values_betta = read_data(experimental_file_betta)

    # Leer los datos de Lab_Gamma
    timestamps_base1_gamma, base_values1_gamma = read_data(base_file1_gamma)
    timestamps_base2_gamma, base_values2_gamma = read_data(base_file2_gamma)
    timestamps_experimental_gamma, experimental_values_gamma = read_data(experimental_file_gamma)

    # Reiniciar todas las estampas de tiempo para Lab_Betta
    start_time_base1_betta = timestamps_base1_betta[0]
    start_time_base2_betta = timestamps_base2_betta[0]
    start_time_experimental_betta = timestamps_experimental_betta[0]
    timestamps_base1_betta = [(ts - start_time_base1_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_base1_betta]
    timestamps_base2_betta = [(ts - start_time_base2_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_base2_betta]
    timestamps_experimental_betta = [(ts - start_time_experimental_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental_betta]

    # Reiniciar todas las estampas de tiempo para Lab_Gamma
    start_time_base1_gamma = timestamps_base1_gamma[0]
    start_time_base2_gamma = timestamps_base2_gamma[0]
    start_time_experimental_gamma = timestamps_experimental_gamma[0]
    timestamps_base1_gamma = [(ts - start_time_base1_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_base1_gamma]
    timestamps_base2_gamma = [(ts - start_time_base2_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_base2_gamma]
    timestamps_experimental_gamma = [(ts - start_time_experimental_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental_gamma]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    base_times1_betta = mdates.date2num(timestamps_base1_betta)
    base_times2_betta = mdates.date2num(timestamps_base2_betta)
    experimental_times_betta = mdates.date2num(timestamps_experimental_betta)

    base_times1_gamma = mdates.date2num(timestamps_base1_gamma)
    base_times2_gamma = mdates.date2num(timestamps_base2_gamma)
    experimental_times_gamma = mdates.date2num(timestamps_experimental_gamma)
    
    # Obtener las triadas para los títulos
    triada_betta = extract_triada(experimental_file_betta)
    triada_gamma = extract_triada(experimental_file_gamma)

    # Usar los nombres descriptivos de las triadas en los títulos
    triad_name_betta = triad_names.get(triada_betta, triada_betta)
    triad_name_gamma = triad_names.get(triada_gamma, triada_gamma)

    # Lab_Betta gráfico
    ax1.plot(base_times1_betta, base_values1_betta, c='black', label=os.path.basename(base_file1_betta))
    ax1.plot(base_times2_betta, base_values2_betta, c='gray', label=os.path.basename(base_file2_betta))
    ax1.plot(experimental_times_betta, experimental_values_betta, c=get_color(experimental_file_betta), label=os.path.basename(experimental_file_betta))
    ax1.set_title(f'Lab_Betta - {shift} - Date: {date_str} - {triad_name_betta}', fontsize=14)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Value')
    ax1.legend()
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    ax1.grid(True)

    # Lab_Gamma gráfico
    ax2.plot(base_times1_gamma, base_values1_gamma, c='black', label=os.path.basename(base_file1_gamma))
    ax2.plot(base_times2_gamma, base_values2_gamma, c='gray', label=os.path.basename(base_file2_gamma))
    ax2.plot(experimental_times_gamma, experimental_values_gamma, c=get_color(experimental_file_gamma), label=os.path.basename(experimental_file_gamma))
    ax2.set_title(f'Lab_Gamma - {shift} - Date: {date_str} - {triad_name_gamma}', fontsize=14)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Value')
    ax2.legend()
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    ax2.grid(True)

    plt.tight_layout()
    
    # Generar un nombre más corto para cada gráfico
    number1_betta = extract_number(base_file1_betta)
    number2_betta = extract_number(base_file2_betta)
    number_exp_betta = extract_number(experimental_file_betta)
    number1_gamma = extract_number(base_file1_gamma)
    number2_gamma = extract_number(base_file2_gamma)
    number_exp_gamma = extract_number(experimental_file_gamma)

    output_filename = (f"Sum_Helicity_Comparison_{number1_betta}_{number2_betta}_{number_exp_betta}_"
                   f"{number1_gamma}_{number2_gamma}_{number_exp_gamma}.png")

    # Determinar la carpeta de salida en función del código de triada del archivo experimental de Lab_Betta
    triad_code_betta = re.search(r'helicity_(\w+)_', experimental_file_betta)
    if triad_code_betta:
        triad_code_betta = triad_code_betta.group(1)
        specific_output_dir = os.path.join(output_dir, triad_code_betta)
    else:
        specific_output_dir = output_dir

    if not os.path.exists(specific_output_dir):
        os.makedirs(specific_output_dir)

    output_filepath = os.path.join(specific_output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    # Extraer triadas de los nombres de los archivos
    baseline_triads = set()
    experimental_triads = set()

    for f in baseline_files:
        match = re.search(r'helicity_(\w+)_', f)
        if match:
            baseline_triads.add(match.group(1))

    for f in experimental_files:
        match = re.search(r'helicity_(\w+)_', f)
        if match:
            experimental_triads.add(match.group(1))

    # Generar combinaciones solo para triadas comunes
    common_triads = baseline_triads.intersection(experimental_triads)
    for triad in common_triads:
        baseline_files_for_triad = [f for f in baseline_files if f"helicity_{triad}_" in f]
        experimental_files_for_triad = [f for f in experimental_files if f"helicity_{triad}_" in f]
        
        # Asegurarse de que hay al menos dos archivos base para cada triada
        if len(baseline_files_for_triad) >= 2:
            for i, base_file1 in enumerate(baseline_files_for_triad):
                for base_file2 in baseline_files_for_triad[i+1:]:
                    for exp_file in experimental_files_for_triad:
                        number1 = int(extract_number(base_file1))
                        number2 = int(extract_number(base_file2))
                        number_exp = int(extract_number(exp_file))
                        if number1 == number_exp == ((number2)-1):
                            combinations.append((base_file1, base_file2, exp_file))
    return combinations

def extract_triada(ruta):
    # Extrae el nombre del archivo de la ruta
    nombre_archivo = os.path.basename(ruta)
    
    # Busca y extrae la triada
    match_triada = re.search(r'(FLB|FRP|PLB|PRB|FRT|PRT|FLT|PLT|RTB|LTB|FRB|FLP)', nombre_archivo)
    triada = match_triada.group(1) if match_triada else None
    
    return triada

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    combinations1 = []  # Definir una lista para combinaciones de Lab_Betta
    combinations2 = []  # Definir una lista para combinaciones de Lab_Gamma

    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color")
            output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Absolute_Helicity_Comp")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Buscar en subcarpetas de triadas
            baseline_files = []
            experimental_files = []

            for triad_code in triad_codes.values():
                # Buscar en subcarpetas basadas en triad_codes
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'sum_absolute_helicity_*mednegra.txt')))
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'sum_absolute_helicity_*med*.txt')))

            if lab == "Lab_Betta":
                combinations1 = generate_combinations(baseline_files, experimental_files)
                combinations1 = list(set(combinations1))
            else:
                combinations2 = generate_combinations(baseline_files, experimental_files)
                combinations2 = list(set(combinations2))

    for base_file1_betta, base_file2_betta, experimental_file_betta in combinations1:
        for base_file1_gamma, base_file2_gamma, experimental_file_gamma in combinations2:
            number1_betta = extract_number(base_file1_betta)
            number2_betta = extract_number(base_file2_betta)
            number_exp_betta = extract_number(experimental_file_betta)
            number1_gamma = extract_number(base_file1_gamma)
            number2_gamma = extract_number(base_file2_gamma)
            number_exp_gamma = extract_number(experimental_file_gamma)
            triada1_betta = extract_triada(base_file1_betta)
            triada2_betta = extract_triada(base_file2_betta)
            triada_exp_betta = extract_triada(experimental_file_betta)
            triada1_gamma = extract_triada(base_file1_gamma)
            triada2_gamma = extract_triada(base_file1_gamma)
            triada_exp_gamma = extract_triada(experimental_file_gamma)
            
            if (number1_betta == number1_gamma and number2_betta == number2_gamma and number_exp_betta == number_exp_gamma and 
                triada1_betta == triada2_betta == triada_exp_betta and 
                triada1_gamma == triada2_gamma == triada_exp_gamma and 
                triada1_betta == triada1_gamma and 
                triada2_betta == triada2_gamma and 
                triada_exp_betta == triada_exp_gamma):
                plot_comparison(base_file1_betta, base_file2_betta, experimental_file_betta,
                            base_file1_gamma, base_file2_gamma, experimental_file_gamma,
                            output_directory, shift, date_str)

if __name__ == "__main__":
    main()

Omitiendo cálculo para Lab_Betta V ya que el directorio no existe.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Absolute_Helicity_Comp\FLT_velocity\Sum_Helicity_Comparison_03_04_03_03_04_03.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Absolute_Helicity_Comp\PLT_velocity\Sum_Helicity_Comparison_03_04_03_03_04_03.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Absolute_Helicity_Comp\FLB_velocity\Sum_Helicity_Comparison_01_02_01_01_02_01.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Absolute_Helicity_Comp\LTB_velocity\Sum_Helicity_Comparison_02_03_02_02_03_02.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Absolute_Helicity_Comp\PLB_velocity\Sum_Helicity_Comparison_03_04_03_03_04_03.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Absolute_Helicity_Comp\FLP_velocity\Sum_Helicity_Comparison_02_03_02_02_03_02.png
Saved plot as 


## 5 
$$
\Huge \text{First Integer Enstrophy Betta vs Gamma}
$$

In [6]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob
import re

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)
                values.append(float(parts[1]))
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow',
        'mednaranja.txt': 'orange'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_comparison(base_file1_betta, base_file2_betta, experimental_file_betta,
                    base_file1_gamma, base_file2_gamma, experimental_file_gamma,
                    output_dir, shift, date_str):
    # Leer los datos de Lab_Betta
    timestamps_base1_betta, base_values1_betta = read_data(base_file1_betta)
    timestamps_base2_betta, base_values2_betta = read_data(base_file2_betta)
    timestamps_experimental_betta, experimental_values_betta = read_data(experimental_file_betta)

    # Leer los datos de Lab_Gamma
    timestamps_base1_gamma, base_values1_gamma = read_data(base_file1_gamma)
    timestamps_base2_gamma, base_values2_gamma = read_data(base_file2_gamma)
    timestamps_experimental_gamma, experimental_values_gamma = read_data(experimental_file_gamma)

    # Reiniciar todas las estampas de tiempo para Lab_Betta
    start_time_base1_betta = timestamps_base1_betta[0]
    start_time_base2_betta = timestamps_base2_betta[0]
    start_time_experimental_betta = timestamps_experimental_betta[0]
    timestamps_base1_betta = [(ts - start_time_base1_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_base1_betta]
    timestamps_base2_betta = [(ts - start_time_base2_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_base2_betta]
    timestamps_experimental_betta = [(ts - start_time_experimental_betta + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental_betta]

    # Reiniciar todas las estampas de tiempo para Lab_Gamma
    start_time_base1_gamma = timestamps_base1_gamma[0]
    start_time_base2_gamma = timestamps_base2_gamma[0]
    start_time_experimental_gamma = timestamps_experimental_gamma[0]
    timestamps_base1_gamma = [(ts - start_time_base1_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_base1_gamma]
    timestamps_base2_gamma = [(ts - start_time_base2_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_base2_gamma]
    timestamps_experimental_gamma = [(ts - start_time_experimental_gamma + datetime.datetime(1900, 1, 1)) for ts in timestamps_experimental_gamma]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    base_times1_betta = mdates.date2num(timestamps_base1_betta)
    base_times2_betta = mdates.date2num(timestamps_base2_betta)
    experimental_times_betta = mdates.date2num(timestamps_experimental_betta)

    base_times1_gamma = mdates.date2num(timestamps_base1_gamma)
    base_times2_gamma = mdates.date2num(timestamps_base2_gamma)
    experimental_times_gamma = mdates.date2num(timestamps_experimental_gamma)
    
    # Obtener las triadas para los títulos
    triada_betta = extract_triada(experimental_file_betta)
    triada_gamma = extract_triada(experimental_file_gamma)

    # Usar los nombres descriptivos de las triadas en los títulos
    triad_name_betta = triad_names.get(triada_betta, triada_betta)
    triad_name_gamma = triad_names.get(triada_gamma, triada_gamma)

    # Lab_Betta gráfico
    ax1.plot(base_times1_betta, base_values1_betta, c='black', label=os.path.basename(base_file1_betta))
    ax1.plot(base_times2_betta, base_values2_betta, c='gray', label=os.path.basename(base_file2_betta))
    ax1.plot(experimental_times_betta, experimental_values_betta, c=get_color(experimental_file_betta), label=os.path.basename(experimental_file_betta))
    ax1.set_title(f'Lab_Betta - {shift} - Date: {date_str} - {triad_name_betta}', fontsize=14)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Value')
    ax1.legend()
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    ax1.grid(True)

    # Lab_Gamma gráfico
    ax2.plot(base_times1_gamma, base_values1_gamma, c='black', label=os.path.basename(base_file1_gamma))
    ax2.plot(base_times2_gamma, base_values2_gamma, c='gray', label=os.path.basename(base_file2_gamma))
    ax2.plot(experimental_times_gamma, experimental_values_gamma, c=get_color(experimental_file_gamma), label=os.path.basename(experimental_file_gamma))
    ax2.set_title(f'Lab_Gamma - {shift} - Date: {date_str} - {triad_name_gamma}', fontsize=14)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Value')
    ax2.legend()
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.gcf().autofmt_xdate()
    ax2.grid(True)

    plt.tight_layout()
    
    # Generar un nombre más corto para cada gráfico
    number1_betta = extract_number(base_file1_betta)
    number2_betta = extract_number(base_file2_betta)
    number_exp_betta = extract_number(experimental_file_betta)
    number1_gamma = extract_number(base_file1_gamma)
    number2_gamma = extract_number(base_file2_gamma)
    number_exp_gamma = extract_number(experimental_file_gamma)

    output_filename = (f"Sum_Enstrophy_Comparison_{number1_betta}_{number2_betta}_{number_exp_betta}_"
                   f"{number1_gamma}_{number2_gamma}_{number_exp_gamma}.png")

    # Determinar la carpeta de salida en función del código de triada del archivo experimental de Lab_Betta
    triad_code_betta = re.search(r'vorticity_(\w+)_', experimental_file_betta)
    if triad_code_betta:
        triad_code_betta = triad_code_betta.group(1)
        specific_output_dir = os.path.join(output_dir, triad_code_betta)
    else:
        specific_output_dir = output_dir

    if not os.path.exists(specific_output_dir):
        os.makedirs(specific_output_dir)

    output_filepath = os.path.join(specific_output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files):
    combinations = []
    # Extraer triadas de los nombres de los archivos
    baseline_triads = set()
    experimental_triads = set()

    for f in baseline_files:
        match = re.search(r'vorticity_(\w+)_', f)
        if match:
            baseline_triads.add(match.group(1))

    for f in experimental_files:
        match = re.search(r'vorticity_(\w+)_', f)
        if match:
            experimental_triads.add(match.group(1))

    # Generar combinaciones solo para triadas comunes
    common_triads = baseline_triads.intersection(experimental_triads)
    for triad in common_triads:
        baseline_files_for_triad = [f for f in baseline_files if f"vorticity_{triad}_" in f]
        experimental_files_for_triad = [f for f in experimental_files if f"vorticity_{triad}_" in f]
        
        # Asegurarse de que hay al menos dos archivos base para cada triada
        if len(baseline_files_for_triad) >= 2:
            for i, base_file1 in enumerate(baseline_files_for_triad):
                for base_file2 in baseline_files_for_triad[i+1:]:
                    for exp_file in experimental_files_for_triad:
                        number1 = int(extract_number(base_file1))
                        number2 = int(extract_number(base_file2))
                        number_exp = int(extract_number(exp_file))
                        if number1 == number_exp == ((number2)-1):
                            combinations.append((base_file1, base_file2, exp_file))
    return combinations

def extract_triada(ruta):
    # Extrae el nombre del archivo de la ruta
    nombre_archivo = os.path.basename(ruta)
    
    # Busca y extrae la triada
    match_triada = re.search(r'(FLB|FRP|PLB|PRB|FRT|PRT|FLT|PLT|RTB|LTB|FRB|FLP)', nombre_archivo)
    triada = match_triada.group(1) if match_triada else None
    
    return triada

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    combinations1 = []  # Definir una lista para combinaciones de Lab_Betta
    combinations2 = []  # Definir una lista para combinaciones de Lab_Gamma

    for lab in labs:
        for shift in shifts:
            lab_dirs = glob.glob(os.path.join(root_folder, f"*{lab}"))
            if not lab_dirs:
                print(f"No se encontró ningún directorio para {lab}.")
                continue
            lab_dir = lab_dirs[0]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline")
            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color")
            output_directory = os.path.join(root_folder, "Data Analysis", "Graphics", "Sum_Enstrophy_Comp")

            if not os.path.exists(baseline_dir) or not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory):
                os.makedirs(output_directory)

            # Buscar en subcarpetas de triadas
            baseline_files = []
            experimental_files = []

            for triad_code in triad_codes.values():
                # Buscar en subcarpetas basadas en triad_codes
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'sum_enstrophy_vorticity_*mednegra.txt')))
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'sum_enstrophy_vorticity_*med*.txt')))

            if lab == "Lab_Betta":
                combinations1 = generate_combinations(baseline_files, experimental_files)
                combinations1 = list(set(combinations1))
            else:
                combinations2 = generate_combinations(baseline_files, experimental_files)
                combinations2 = list(set(combinations2))

    for base_file1_betta, base_file2_betta, experimental_file_betta in combinations1:
        for base_file1_gamma, base_file2_gamma, experimental_file_gamma in combinations2:
            number1_betta = extract_number(base_file1_betta)
            number2_betta = extract_number(base_file2_betta)
            number_exp_betta = extract_number(experimental_file_betta)
            number1_gamma = extract_number(base_file1_gamma)
            number2_gamma = extract_number(base_file2_gamma)
            number_exp_gamma = extract_number(experimental_file_gamma)
            triada1_betta = extract_triada(base_file1_betta)
            triada2_betta = extract_triada(base_file2_betta)
            triada_exp_betta = extract_triada(experimental_file_betta)
            triada1_gamma = extract_triada(base_file1_gamma)
            triada2_gamma = extract_triada(base_file1_gamma)
            triada_exp_gamma = extract_triada(experimental_file_gamma)
            
            if (number1_betta == number1_gamma and number2_betta == number2_gamma and number_exp_betta == number_exp_gamma and triada1_betta == triada2_betta == triada_exp_betta and 
                triada1_gamma == triada2_gamma == triada_exp_gamma and 
                triada1_betta == triada1_gamma and 
                triada2_betta == triada2_gamma and 
                triada_exp_betta == triada_exp_gamma):
                plot_comparison(base_file1_betta, base_file2_betta, experimental_file_betta,
                            base_file1_gamma, base_file2_gamma, experimental_file_gamma,
                            output_directory, shift, date_str)

if __name__ == "__main__":
    main()

Omitiendo cálculo para Lab_Betta V ya que el directorio no existe.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Enstrophy_Comp\FLP_velocity\Sum_Enstrophy_Comparison_01_02_01_01_02_01.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Enstrophy_Comp\RTB_velocity\Sum_Enstrophy_Comparison_02_03_02_02_03_02.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Enstrophy_Comp\FLP_velocity\Sum_Enstrophy_Comparison_04_05_04_04_05_04.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Enstrophy_Comp\FLP_velocity\Sum_Enstrophy_Comparison_03_04_03_03_04_03.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Enstrophy_Comp\FRP_velocity\Sum_Enstrophy_Comparison_03_04_03_03_04_03.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\Sum_Enstrophy_Comp\FLP_velocity\Sum_Enstrophy_Comparison_02_03_02_02_03_02.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Graphics\

## 6


$$
    \Huge \text{3D Curl Comparisons }
$$

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime
import os
import glob
from matplotlib.animation import FuncAnimation

# Función para leer datos del sensor
def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

# Función para comparar y graficar los laboratorios
def plot_comparison_labs(baseline_file_betta, exp_file_betta, color_betta, baseline_file_gamma, exp_file_gamma, color_gamma, output_directory, shift, date_str):
    try:
        _, baseline_values_betta = read_data(baseline_file_betta)
        _, exp_values_betta = read_data(exp_file_betta)
        _, baseline_values_gamma = read_data(baseline_file_gamma)
        _, exp_values_gamma = read_data(exp_file_gamma)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Vespertino" if shift == "V" else "Matutino"

    fig = plt.figure(figsize=(24, 10))

    # Subgráfica para el laboratorio Betta
    ax_betta = fig.add_subplot(121, projection='3d')
    ax_betta.scatter(baseline_values_betta[:, 0], baseline_values_betta[:, 1], baseline_values_betta[:, 2], c='black', label=os.path.basename(baseline_file_betta), s=50, alpha=0.6)
    ax_betta.scatter(exp_values_betta[:, 0], exp_values_betta[:, 1], exp_values_betta[:, 2], c=color_betta, label=os.path.basename(exp_file_betta), s=50, alpha=0.6)
    ax_betta.set_title(f'Lab Betta - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax_betta.set_xlabel('X', fontsize=15)
    ax_betta.set_ylabel('Y', fontsize=15)
    ax_betta.set_zlabel('Z', fontsize=15)
    legend_betta = ax_betta.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_betta.legend_handles:
        handle.set_sizes([300])

    # Subgráfica para el laboratorio Gamma
    ax_gamma = fig.add_subplot(122, projection='3d')
    ax_gamma.scatter(baseline_values_gamma[:, 0], baseline_values_gamma[:, 1], baseline_values_gamma[:, 2], c='black', label=os.path.basename(baseline_file_gamma), s=50, alpha=0.6)
    ax_gamma.scatter(exp_values_gamma[:, 0], exp_values_gamma[:, 1], exp_values_gamma[:, 2], c=color_gamma, label=os.path.basename(exp_file_gamma), s=50, alpha=0.6)
    ax_gamma.set_title(f'Lab Gamma - {shift_label} Shift - Date: {date_str}', fontsize=20)
    ax_gamma.set_xlabel('X', fontsize=15)
    ax_gamma.set_ylabel('Y', fontsize=15)
    ax_gamma.set_zlabel('Z', fontsize=15)
    legend_gamma = ax_gamma.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_gamma.legend_handles:
        handle.set_sizes([300])

    def update(frame):
        ax_betta.view_init(elev=10, azim=frame)
        ax_gamma.view_init(elev=10, azim=frame)
        return fig,

    output_filename = f"{os.path.basename(baseline_file_betta).replace('mednegra', 'comparison_betta_vs_gamma')}_{os.path.basename(exp_file_betta)}"
    output_filepath = os.path.join(output_directory, output_filename)

    ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=100, blit=False)
    ani.save(f'{output_filepath}.gif', writer='pillow', fps=10)
    plt.close(fig)

# Función principal para procesar los archivos de ambos laboratorios
def main(root_folder):
    date_str = os.path.basename(root_folder)
    date_str = date_str.replace("-2", "").strip()
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for shift in shifts:
        lab_betta_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Betta"))
        lab_gamma_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Gamma"))

        if not lab_betta_dirs:
            print("No se encontró el directorio para Lab Betta.")
            continue
        if not lab_gamma_dirs:
            print("No se encontró el directorio para Lab Gamma.")
            continue

        lab_betta_dir = lab_betta_dirs[0]
        lab_gamma_dir = lab_gamma_dirs[0]

        betta_baseline_dir = os.path.join(lab_betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Curl_Baseline")
        betta_exp_dir = os.path.join(lab_betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

        gamma_baseline_dir = os.path.join(lab_gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Curl_Baseline")
        gamma_exp_dir = os.path.join(lab_gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

        output_directory = os.path.join(root_folder, "Data Analysis", "3D Curl Comparissons")
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # Omitir si alguna de las carpetas requeridas no existe
        if not os.path.exists(betta_baseline_dir) or not os.path.exists(betta_exp_dir) or not os.path.exists(gamma_baseline_dir) or not os.path.exists(gamma_exp_dir):
            print(f"Omitiendo cálculo para el turno {shift} ya que uno o más directorios no existen.")
            continue

        betta_baseline_files = [f for f in os.listdir(betta_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
        gamma_baseline_files = [f for f in os.listdir(gamma_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]

        betta_exp_files = [f for f in os.listdir(betta_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]
        gamma_exp_files = [f for f in os.listdir(gamma_exp_dir) if any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]

        # Verificar que hay archivos para procesar
        if not betta_baseline_files or not gamma_baseline_files or not betta_exp_files or not gamma_exp_files:
            print(f"Omitiendo cálculo para el turno {shift} ya que no se encontraron archivos de base o experimentales en uno de los laboratorios.")
            continue

        combinations_betta = generate_combinations(betta_baseline_files, betta_exp_files)
        combinations_gamma = generate_combinations(gamma_baseline_files, gamma_exp_files)

        # Asumiendo que las combinaciones tienen el mismo número de pares para Betta y Gamma
        for (base_betta, exp_betta), (base_gamma, exp_gamma) in zip(combinations_betta, combinations_gamma):
            baseline_file_betta = os.path.join(betta_baseline_dir, base_betta)
            experimental_file_betta = os.path.join(betta_exp_dir, exp_betta)
            baseline_file_gamma = os.path.join(gamma_baseline_dir, base_gamma)
            experimental_file_gamma = os.path.join(gamma_exp_dir, exp_gamma)

            if os.path.exists(baseline_file_betta) and os.path.exists(experimental_file_betta) and os.path.exists(baseline_file_gamma) and os.path.exists(experimental_file_gamma):
                exp_suffix_betta = ''.join([i for i in os.path.basename(experimental_file_betta).split('curl_')[-1] if not i.isdigit()])
                exp_suffix_gamma = ''.join([i for i in os.path.basename(experimental_file_gamma).split('curl_')[-1] if not i.isdigit()])
                color_betta = color_map.get(exp_suffix_betta, 'black')
                color_gamma = color_map.get(exp_suffix_gamma, 'black')
                plot_comparison_labs(baseline_file_betta, experimental_file_betta, color_betta, baseline_file_gamma, experimental_file_gamma, color_gamma, output_directory, shift, date_str)

# Generar combinaciones de archivos de línea base y experimentales
def generate_combinations(baseline_files, exp_files):
    # Mantener la misma lógica que genera combinaciones relevantes como en el código original
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in exp_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            if exp_num == base_num or exp_num == base_num - 1:
                combinations.append((base_file, exp_file))
    return combinations

if __name__ == "__main__":
    main(root_folder)


Omitiendo cálculo para el turno V ya que uno o más directorios no existen.


## 7

$$
 \Huge \text{Individual 3D Curl Baseline vs Experimental Color}
$$

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime
import os
import glob
from matplotlib.animation import FuncAnimation

# Diccionario con los nombres de las triadas
triad_names = {
    "FRT": "Frontal-Derecho-Superior",
    "PLB": "Trasero-Izquierdo-Inferior",
    "FLT": "Frontal-Izquierdo-Superior",
    "PRB": "Trasero-Derecho-Inferior",
    "FRB": "Frontal-Derecho-Inferior",
    "PLT": "Trasero-Izquierdo-Superior",
    "FLB": "Frontal-Izquierdo-Inferior",
    "PRT": "Trasero-Derecho-Superior",
    "RTB": "Derecho-Superior-Inferior",
    "FLP": "Frontal-Izquierdo-Trasero",
    "LTB": "Izquierdo-Superior-Inferior",
    "FRP": "Frontal-Derecho-Trasero"
}

# Función para leer datos del sensor
def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                try:
                    timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                    timestamps.append(timestamp)
                    values.append([float(num) for num in parts[1].strip('()').split(',')])
                except ValueError as e:
                    print(f"Error processing line in file {filepath}: {line}\n{e}")
    return timestamps, np.array(values)

# Función para extraer el código de triada desde el nombre del archivo
def extract_triad_code(filename):
    # Asumimos que el código de la triada está en el nombre del archivo
    for code in triad_names:
        if code in filename:
            return code
    return "Desconocido"

# Función para comparar y graficar los laboratorios
def plot_comparison_labs(baseline_file_betta, exp_file_betta, color_betta, baseline_file_gamma, exp_file_gamma, color_gamma, output_directory, shift, date_str):
    try:
        _, baseline_values_betta = read_data(baseline_file_betta)
        _, exp_values_betta = read_data(exp_file_betta)
        _, baseline_values_gamma = read_data(baseline_file_gamma)
        _, exp_values_gamma = read_data(exp_file_gamma)
    except Exception as e:
        print(f"Error reading data files:\n{e}")
        return

    shift_label = "Vespertino" if shift == "V" else "Matutino"

    # Extraer los códigos de las triadas desde los nombres de los archivos
    triad_code_betta = extract_triad_code(baseline_file_betta)
    triad_code_gamma = extract_triad_code(baseline_file_gamma)

    triad_name_betta = triad_names.get(triad_code_betta, "Desconocido")
    triad_name_gamma = triad_names.get(triad_code_gamma, "Desconocido")

    fig = plt.figure(figsize=(24, 10))

    # Subgráfica para el laboratorio Betta
    ax_betta = fig.add_subplot(121, projection='3d')
    ax_betta.scatter(baseline_values_betta[:, 0], baseline_values_betta[:, 1], baseline_values_betta[:, 2], c='black', label=os.path.basename(baseline_file_betta), s=50, alpha=0.6)
    ax_betta.scatter(exp_values_betta[:, 0], exp_values_betta[:, 1], exp_values_betta[:, 2], c=color_betta, label=os.path.basename(exp_file_betta), s=50, alpha=0.6)
    ax_betta.set_title(f'Lab Betta - {shift_label} Shift - {triad_name_betta} - Date: {date_str}', fontsize=20)
    ax_betta.set_xlabel('X', fontsize=15)
    ax_betta.set_ylabel('Y', fontsize=15)
    ax_betta.set_zlabel('Z', fontsize=15)
    legend_betta = ax_betta.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_betta.legend_handles:
        handle.set_sizes([300])

    # Subgráfica para el laboratorio Gamma
    ax_gamma = fig.add_subplot(122, projection='3d')
    ax_gamma.scatter(baseline_values_gamma[:, 0], baseline_values_gamma[:, 1], baseline_values_gamma[:, 2], c='black', label=os.path.basename(baseline_file_gamma), s=50, alpha=0.6)
    ax_gamma.scatter(exp_values_gamma[:, 0], exp_values_gamma[:, 1], exp_values_gamma[:, 2], c=color_gamma, label=os.path.basename(exp_file_gamma), s=50, alpha=0.6)
    ax_gamma.set_title(f'Lab Gamma - {shift_label} Shift - {triad_name_gamma} - Date: {date_str}', fontsize=20)
    ax_gamma.set_xlabel('X', fontsize=15)
    ax_gamma.set_ylabel('Y', fontsize=15)
    ax_gamma.set_zlabel('Z', fontsize=15)
    legend_gamma = ax_gamma.legend(fontsize=14, loc='upper left', frameon=True, framealpha=0.9)
    for handle in legend_gamma.legend_handles:
        handle.set_sizes([300])

    def update(frame):
        ax_betta.view_init(elev=10, azim=frame)
        ax_gamma.view_init(elev=10, azim=frame)
        return fig,

    output_filename = f"{os.path.basename(baseline_file_betta).replace('mednegra', 'comparison_betta_vs_gamma')}_{os.path.basename(exp_file_betta)}"
    output_filepath = os.path.join(output_directory, output_filename)

    ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), interval=100, blit=False)
    ani.save(f'{output_filepath}.gif', writer='pillow', fps=10)
    plt.close(fig)

# Función principal para procesar los archivos de ambos laboratorios
def main(root_folder):
    date_str = os.path.basename(root_folder)
    date_str = date_str.replace("-2", "").strip()
    shifts = ["V", "M"]

    color_map = {
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for shift in shifts:
        lab_betta_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Betta"))
        lab_gamma_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Gamma"))

        if not lab_betta_dirs:
            print("No se encontró el directorio para Lab Betta.")
            continue
        if not lab_gamma_dirs:
            print("No se encontró el directorio para Lab Gamma.")
            continue

        lab_betta_dir = lab_betta_dirs[0]
        lab_gamma_dir = lab_gamma_dirs[0]

        betta_baseline_dir = os.path.join(lab_betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Curl_Baseline")
        betta_exp_dir = os.path.join(lab_betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

        gamma_baseline_dir = os.path.join(lab_gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Curl_Baseline")
        gamma_exp_dir = os.path.join(lab_gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Curl_Experimental_Color")

        output_directory = os.path.join(root_folder, "Data Analysis", "3D Curl Comparissons")
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # Omitir si alguna de las carpetas requeridas no existe
        if not os.path.exists(betta_baseline_dir) or not os.path.exists(betta_exp_dir) or not os.path.exists(gamma_baseline_dir) or not os.path.exists(gamma_exp_dir):
            print(f"Omitiendo cálculo para el turno {shift} ya que uno o más directorios no existen.")
            continue

        betta_baseline_files = [f for f in os.listdir(betta_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]
        gamma_baseline_files = [f for f in os.listdir(gamma_baseline_dir) if f.endswith('mednegra.txt') and "00" not in f]

        betta_exp_files = [f for f in os.listdir(betta_exp_dir) if f.endswith('.txt') and not f.endswith('mednegra.txt') and "00" not in f]
        gamma_exp_files = [f for f in os.listdir(gamma_exp_dir) if f.endswith('.txt') and not f.endswith('mednegra.txt') and "00" not in f]

        # Procesar cada archivo correspondiente
        for betta_baseline_file in betta_baseline_files:
            for exp_file_betta in betta_exp_files:
                for gamma_baseline_file in gamma_baseline_files:
                    for exp_file_gamma in gamma_exp_files:
                        betta_baseline_path = os.path.join(betta_baseline_dir, betta_baseline_file)
                        exp_file_betta_path = os.path.join(betta_exp_dir, exp_file_betta)
                        gamma_baseline_path = os.path.join(gamma_baseline_dir, gamma_baseline_file)
                        exp_file_gamma_path = os.path.join(gamma_exp_dir, exp_file_gamma)

                        betta_color = color_map.get(exp_file_betta, "blue")
                        gamma_color = color_map.get(exp_file_gamma, "blue")

                        plot_comparison_labs(betta_baseline_path, exp_file_betta_path, betta_color, gamma_baseline_path, exp_file_gamma_path, gamma_color, output_directory, shift, date_str)

## 8

$$
  \Huge \text{Velocity Field  Baseline  Nivel}
$$

In [9]:
#Curvas de Nivel Experimentals
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import glob
import re
import datetime

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    """
    Extrae datos del archivo y los devuelve como arrays de x, y, z.
    """
    timestamps = []
    values_x = []
    values_y = []
    values_z = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                # Parsear el timestamp
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)

                # Remover paréntesis antes de convertir a float
                coords_str = parts[1].strip().replace('(', '').replace(')', '')
                coords = list(map(float, coords_str.split(',')))  # Convertir a float
                values_x.append(coords[0])
                values_y.append(coords[1])
                values_z.append(coords[2])
    
    return np.array(values_x), np.array(values_y), np.array(values_z)

def plot_contour(x, y, z, output_file):
    """
    Genera y guarda la gráfica de curvas de nivel.
    """
    # Crear una malla de puntos (grid) para la interpolación
    grid_x, grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]

    # Interpolación de los valores de z en la malla
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

    # Crear la gráfica de curvas de nivel
    plt.figure(figsize=(10, 8))
    contour = plt.contourf(grid_x, grid_y, grid_z, cmap='viridis')
    plt.colorbar(contour)
    plt.title('Curvas de Nivel')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Guardar la gráfica
    plt.savefig(output_file)
    plt.close()

def process_files(file_list, output_dir):
    """
    Procesa una lista de archivos, genera y guarda gráficos de curvas de nivel para cada uno.
    """
    for file_path in file_list:
        x, y, z = read_data(file_path)
        triada = next((code for code, value in triad_codes.items() if value in os.path.basename(file_path)), None)
        if not triada:
            continue

        triada_code = triad_codes.get(triada)
        output_dir_triade = os.path.join(output_dir, triada_code)

        if not os.path.exists(output_dir_triade):
            os.makedirs(output_dir_triade)

        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join(output_dir_triade, f"{file_name}_contour.png")
        plot_contour(x, y, z, output_file)

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    for shift in shifts:
        lab_dirs = [glob.glob(os.path.join(root_folder, f"*{lab}")) for lab in labs]

        if len(lab_dirs[0]) == 0 or len(lab_dirs[1]) == 0:
            print("No se encontraron directorios para uno o ambos laboratorios.")
            continue

        lab_dirs = [lab_dirs[i][0] for i in range(len(labs))]

        for lab_idx, lab in enumerate(labs):
            lab_dir = lab_dirs[lab_idx]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            baseline_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Velocity_Baseline")
            
            output_directory_baseline = os.path.join(root_folder, "Data Analysis", "Graphics", "Velocity_Baseline_Nivel", lab)

            if not os.path.exists(baseline_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory_baseline):
                os.makedirs(output_directory_baseline)

            # Buscar archivos en directorios de triadas
            baseline_files = []

            for triad_code in triad_codes.values():
                baseline_files.extend(glob.glob(os.path.join(baseline_dir, triad_code, 'velocity_*med*.txt')))

            # Procesar archivos de línea base
            process_files(baseline_files, output_directory_baseline)

if __name__ == "__main__":
    main()


Omitiendo cálculo para Lab_Betta V ya que el directorio no existe.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.


## 9

$$
  \Huge \text{Velocity Field  MEI  Nivel}
$$

In [10]:
#Curvas de Nivel Baselines
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import glob
import re
import datetime

# Diccionario de triadas y códigos
triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

def extract_number(filename):
    match = re.search(r'velocity_(\d+)med', filename)
    if match:
        return match.group(1)
    return None

def read_data(filepath):
    """
    Extrae datos del archivo y los devuelve como arrays de x, y, z.
    """
    timestamps = []
    values_x = []
    values_y = []
    values_z = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                # Parsear el timestamp
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)

                # Remover paréntesis antes de convertir a float
                coords_str = parts[1].strip().replace('(', '').replace(')', '')
                coords = list(map(float, coords_str.split(',')))  # Convertir a float
                values_x.append(coords[0])
                values_y.append(coords[1])
                values_z.append(coords[2])
    
    return np.array(values_x), np.array(values_y), np.array(values_z)

def plot_contour(x, y, z, output_file):
    """
    Genera y guarda la gráfica de curvas de nivel.
    """
    # Crear una malla de puntos (grid) para la interpolación
    grid_x, grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]

    # Interpolación de los valores de z en la malla
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

    # Crear la gráfica de curvas de nivel
    plt.figure(figsize=(10, 8))
    contour = plt.contourf(grid_x, grid_y, grid_z, cmap='viridis')
    plt.colorbar(contour)
    plt.title('Curvas de Nivel')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Guardar la gráfica
    plt.savefig(output_file)
    plt.close()

def process_files(file_list, output_dir):
    """
    Procesa una lista de archivos, genera y guarda gráficos de curvas de nivel para cada uno.
    """
    for file_path in file_list:
        x, y, z = read_data(file_path)
        triada = next((code for code, value in triad_codes.items() if value in os.path.basename(file_path)), None)
        if not triada:
            continue

        triada_code = triad_codes.get(triada)
        output_dir_triade = os.path.join(output_dir, triada_code)

        if not os.path.exists(output_dir_triade):
            os.makedirs(output_dir_triade)

        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join(output_dir_triade, f"{file_name}_contour.png")
        plot_contour(x, y, z, output_file)

def main():
    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    for shift in shifts:
        lab_dirs = [glob.glob(os.path.join(root_folder, f"*{lab}")) for lab in labs]

        if len(lab_dirs[0]) == 0 or len(lab_dirs[1]) == 0:
            print("No se encontraron directorios para uno o ambos laboratorios.")
            continue

        lab_dirs = [lab_dirs[i][0] for i in range(len(labs))]

        for lab_idx, lab in enumerate(labs):
            lab_dir = lab_dirs[lab_idx]
            date_str = os.path.basename(lab_dir).split(' ')[0]

            exp_dir = os.path.join(lab_dir, f"{date_str}.{shift} - {lab}", "Data Analysis", "Processing Data", "Velocity_Experimental_Color")
            
            output_directory_experimental = os.path.join(root_folder, "Data Analysis", "Graphics", "Velocity_Experimental_Nivel", lab)

            if not os.path.exists(exp_dir):
                print(f"Omitiendo cálculo para {lab} {shift} ya que el directorio no existe.")
                continue

            if not os.path.exists(output_directory_experimental):
                os.makedirs(output_directory_experimental)

            # Buscar archivos en directorios de triadas
            experimental_files = []

            for triad_code in triad_codes.values():
                experimental_files.extend(glob.glob(os.path.join(exp_dir, triad_code, 'velocity_*med*.txt')))

            # Procesar archivos experimentales
            process_files(experimental_files, output_directory_experimental)

if __name__ == "__main__":
    main()

Omitiendo cálculo para Lab_Betta V ya que el directorio no existe.
Omitiendo cálculo para Lab_Gamma V ya que el directorio no existe.


## 10

$$
\Huge \text{Comparison Moduli before and after for MEI}
$$

In [11]:
#--------------------------------------------------------------------COMPARACIÓN MODULO LAB_BETTA VS MODULO LAB_GAMMA----------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import os
import glob

def read_data(filepath):
    timestamps = []
    values = []
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                timestamp = datetime.datetime.strptime(parts[0], "%H:%M:%S.%f")
                timestamps.append(timestamp)
                values.append(float(parts[1]))
    return timestamps, np.array(values)

def get_color(filename):
    color_map = {
        'mednegra.txt': 'black',
        'medroja.txt': 'red',
        'medmorada.txt': 'purple',
        'medazul.txt': 'blue',
        'medverde.txt': 'green',
        'medamarilla.txt': 'yellow'
    }
    for suffix, color in color_map.items():
        if suffix in filename:
            return color
    return 'gray'  # Default color if no match found

def plot_triad_comparison(betta_baseline_file, betta_experimental_file, betta_baseline_file2,
                          gamma_baseline_file, gamma_experimental_file, gamma_baseline_file2,
                          output_dir, shift, date_str):
    # Leer datos de los archivos de ambos laboratorios
    timestamps_betta_baseline, betta_baseline_values = read_data(betta_baseline_file)
    timestamps_betta_experimental, betta_experimental_values = read_data(betta_experimental_file)
    timestamps_betta_baseline2, betta_baseline_values2 = read_data(betta_baseline_file2)
    
    timestamps_gamma_baseline, gamma_baseline_values = read_data(gamma_baseline_file)
    timestamps_gamma_experimental, gamma_experimental_values = read_data(gamma_experimental_file)
    timestamps_gamma_baseline2, gamma_baseline_values2 = read_data(gamma_baseline_file2)

    # Reiniciar todas las estampas de tiempo
    start_time_betta_baseline = timestamps_betta_baseline[0]
    start_time_betta_experimental = timestamps_betta_experimental[0]
    start_time_betta_baseline2 = timestamps_betta_baseline2[0]
    
    start_time_gamma_baseline = timestamps_gamma_baseline[0]
    start_time_gamma_experimental = timestamps_gamma_experimental[0]
    start_time_gamma_baseline2 = timestamps_gamma_baseline2[0]
    
    timestamps_betta_baseline = [(ts - start_time_betta_baseline + datetime.datetime(1900, 1, 1)) for ts in timestamps_betta_baseline]
    timestamps_betta_experimental = [(ts - start_time_betta_experimental + datetime.datetime(1900, 1, 1)) for ts in timestamps_betta_experimental]
    timestamps_betta_baseline2 = [(ts - start_time_betta_baseline2 + datetime.datetime(1900, 1, 1)) for ts in timestamps_betta_baseline2]

    timestamps_gamma_baseline = [(ts - start_time_gamma_baseline + datetime.datetime(1900, 1, 1)) for ts in timestamps_gamma_baseline]
    timestamps_gamma_experimental = [(ts - start_time_gamma_experimental + datetime.datetime(1900, 1, 1)) for ts in timestamps_gamma_experimental]
    timestamps_gamma_baseline2 = [(ts - start_time_gamma_baseline2 + datetime.datetime(1900, 1, 1)) for ts in timestamps_gamma_baseline2]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

    # Graficar Lab Betta con tres series de tiempo
    betta_baseline_times = mdates.date2num(timestamps_betta_baseline)
    betta_experimental_times = mdates.date2num(timestamps_betta_experimental)
    betta_baseline_times2 = mdates.date2num(timestamps_betta_baseline2)

    ax1.plot(betta_baseline_times, betta_baseline_values, c='gray', label=os.path.basename(betta_baseline_file))
    ax1.plot(betta_experimental_times, betta_experimental_values, c=get_color(betta_experimental_file), label=os.path.basename(betta_experimental_file))
    ax1.plot(betta_baseline_times2, betta_baseline_values2, c='black', label=os.path.basename(betta_baseline_file2))
    ax1.set_title(f'Lab Betta - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Magnitude')
    ax1.legend()
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # Graficar Lab Gamma con tres series de tiempo
    gamma_baseline_times = mdates.date2num(timestamps_gamma_baseline)
    gamma_experimental_times = mdates.date2num(timestamps_gamma_experimental)
    gamma_baseline_times2 = mdates.date2num(timestamps_gamma_baseline2)

    ax2.plot(gamma_baseline_times, gamma_baseline_values, c='gray', label=os.path.basename(gamma_baseline_file))
    ax2.plot(gamma_experimental_times, gamma_experimental_values, c=get_color(gamma_experimental_file), label=os.path.basename(gamma_experimental_file))
    ax2.plot(gamma_baseline_times2, gamma_baseline_values2, c='black', label=os.path.basename(gamma_baseline_file2))
    ax2.set_title(f'Lab Gamma - Shift: {"Vespertino" if shift == "V" else "Matutino"} - Date: {date_str}', fontsize=14)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Magnitude')
    ax2.legend()
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    
    plt.gcf().autofmt_xdate()
    plt.tight_layout()

    # Generar un nombre único para cada gráfico
    output_filename = f"Comparison_Triad_Betta_vs_Gamma_{os.path.basename(betta_baseline_file).split('.')[0]}_VS_{os.path.basename(gamma_baseline_file).split('.')[0]}.png"
    output_filepath = os.path.join(output_dir, output_filename)
    plt.savefig(output_filepath)
    print(f"Saved plot as {output_filepath}")
    plt.close(fig)

def generate_combinations(baseline_files, experimental_files, baseline_files2):
    combinations = []
    for base_file in baseline_files:
        base_num = int(base_file.split('med')[0][-2:])
        for exp_file in experimental_files:
            exp_num = int(exp_file.split('med')[0][-2:])
            for base_file2 in baseline_files2:
                base_num2 = int(base_file2.split('med')[0][-2:])
                if exp_num == base_num - 1 and exp_num == base_num2:
                    combinations.append((base_file, exp_file, base_file2))
    return combinations

def main():

    labs = ["Lab_Betta", "Lab_Gamma"]
    shifts = ["V", "M"]

    color_map = {
        "mednegra.txt": "black",
        "medroja.txt": "red",
        "medmorada.txt": "purple",
        "medazul.txt": "blue",
        "medverde.txt": "green",
        "medamarilla.txt": "yellow"
    }

    for shift in shifts:
        betta_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Betta"))
        gamma_dirs = glob.glob(os.path.join(root_folder, f"*Lab_Gamma"))
        
        if not betta_dirs or not gamma_dirs:
            print(f"No se encontró ningún directorio para {shift}.")
            continue

        betta_dir = betta_dirs[0]
        gamma_dir = gamma_dirs[0]

        date_str = os.path.basename(betta_dir).split(' ')[0]

        betta_baseline_dir = os.path.join(betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
        betta_exp_dir = os.path.join(betta_dir, f"{date_str}.{shift} - Lab_Betta", "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")

        gamma_baseline_dir = os.path.join(gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
        gamma_exp_dir = os.path.join(gamma_dir, f"{date_str}.{shift} - Lab_Gamma", "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")

        output_directory = os.path.join(root_folder, "Data Analysis", "Moduli Comparisons")
        os.makedirs(output_directory, exist_ok=True)

        if not os.path.exists(betta_baseline_dir) or not os.path.exists(betta_exp_dir) or not os.path.exists(gamma_baseline_dir) or not os.path.exists(gamma_exp_dir):
            print(f"Omitiendo cálculo para el turno {shift}: directorio basal o experimental no existe para uno de los laboratorios.")
            continue

        betta_baseline_files = [f for f in os.listdir(betta_baseline_dir) if f.startswith('magnitude_') and f.endswith('mednegra.txt') and "00" not in f]
        betta_experimental_files = [f for f in os.listdir(betta_exp_dir) if f.startswith('magnitude_') and any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]
        betta_baseline_files2 = [f for f in os.listdir(betta_baseline_dir) if f.startswith('magnitude_') and f.endswith('mednegra.txt') and "00" not in f and f != betta_baseline_files]

        gamma_baseline_files = [f for f in os.listdir(gamma_baseline_dir) if f.startswith('magnitude_') and f.endswith('mednegra.txt') and "00" not in f]
        gamma_experimental_files = [f for f in os.listdir(gamma_exp_dir) if f.startswith('magnitude_') and any(f.endswith(suffix) for suffix in color_map.keys()) and "00" not in f]
        gamma_baseline_files2 = [f for f in os.listdir(gamma_baseline_dir) if f.startswith('magnitude_') and f.endswith('mednegra.txt') and "00" not in f and f != gamma_baseline_files]

        betta_combinations = generate_combinations(betta_baseline_files, betta_experimental_files, betta_baseline_files2)
        gamma_combinations = generate_combinations(gamma_baseline_files, gamma_experimental_files, gamma_baseline_files2)

        for (betta_base_file, betta_exp_file, betta_base_file2), (gamma_base_file, gamma_exp_file, gamma_base_file2) in zip(betta_combinations, gamma_combinations):
            betta_baseline_file = os.path.join(betta_baseline_dir, betta_base_file)
            betta_experimental_file = os.path.join(betta_exp_dir, betta_exp_file)
            betta_baseline_file2 = os.path.join(betta_baseline_dir, betta_base_file2)

            gamma_baseline_file = os.path.join(gamma_baseline_dir, gamma_base_file)
            gamma_experimental_file = os.path.join(gamma_exp_dir, gamma_exp_file)
            gamma_baseline_file2 = os.path.join(gamma_baseline_dir, gamma_base_file2)

            if all(os.path.exists(file) for file in [betta_baseline_file, betta_experimental_file, betta_baseline_file2, gamma_baseline_file, gamma_experimental_file, gamma_baseline_file2]):
                plot_triad_comparison(betta_baseline_file, betta_experimental_file, betta_baseline_file2,
                                      gamma_baseline_file, gamma_experimental_file, gamma_baseline_file2,
                                      output_directory, shift, date_str)

if __name__ == "__main__":
    main()


Omitiendo cálculo para el turno V: directorio basal o experimental no existe para uno de los laboratorios.
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Moduli Comparisons\Comparison_Triad_Betta_vs_Gamma_magnitude_02mednegra_VS_magnitude_02mednegra.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Moduli Comparisons\Comparison_Triad_Betta_vs_Gamma_magnitude_03mednegra_VS_magnitude_03mednegra.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Moduli Comparisons\Comparison_Triad_Betta_vs_Gamma_magnitude_04mednegra_VS_magnitude_04mednegra.png
Saved plot as E:\Pruebas\14Sep24\Data Analysis\Moduli Comparisons\Comparison_Triad_Betta_vs_Gamma_magnitude_05mednegra_VS_magnitude_05mednegra.png


## 11

$$
    \Huge \text{Comparison Derivatives Histograms}
$$

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re

def read_sensor_data(filepath):
    data = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return np.array(data).T

def analyze_folder(root_folder):
    data_per_file = {}
    print(f"Analizando la carpeta raíz: {root_folder}\n")
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d != "Data Analysis"]
        print(f"Subdirectorio: {root}")
        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt")) and not filename.startswith("00"):
                file_path = os.path.join(root, filename)
                data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = data
                    print(f"   Encontrado y procesado: {filename}")
                else:
                    print(f"   Archivo vacío o no procesable: {filename}")
            else:
                print(f"   Archivo ignorado: {filename}")
        print("\n")
    return data_per_file

def smooth_data(data, window_length=10, polyorder=3):
    if len(data) < window_length:
        window_length = len(data) - (len(data) % 2 == 0)
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

def plot_comparison_grid(data_per_file, output_folder, title_info, lab):
    suffix_to_color = {
        "negra.txt": "black",
        "roja.txt": "red",
        "morada.txt": "purple",
        "azul.txt": "blue",
        "verde.txt": "green",
        "amarilla.txt": "yellow"
    }
    component_names = ["FRONTAL", "TRASERO", "IZQUIERDA", "DERECHA", "SUPERIOR", "INFERIOR"]

    min_length = min((len(data[0]) for data in data_per_file.values() if data.size > 0), default=None)
    if min_length is None:
        print("No se encontraron datos válidos para procesar.")
        return

    for base_key in sorted(data_per_file.keys()):
        if "negra.txt" in base_key:
            n = int(base_key.split('med')[0])
            base_data_before = data_per_file[base_key][:, :min_length]
            for exp_suffix in ["roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt"]:
                exp_key = f"{n:02d}med{exp_suffix}"
                base_key_after = f"{n+1:02d}mednegra.txt"
                
                if exp_key in data_per_file and base_key_after in data_per_file:
                    exp_data = data_per_file[exp_key][:, :min_length]
                    base_data_after = data_per_file[base_key_after][:, :min_length]

                    fig, axes = plt.subplots(6, 2, figsize=(40, 60), dpi=300)
                    for i, component in enumerate(component_names):
                        ax_left = axes[i, 0]
                        ax_right = axes[i, 1]
                        smoothed_base_before = smooth_data(base_data_before[i])
                        smoothed_exp = smooth_data(exp_data[i])
                        smoothed_base_after = smooth_data(base_data_after[i])
                        derivative_base_before = np.gradient(smoothed_base_before)
                        derivative_exp = np.gradient(smoothed_exp)
                        derivative_base_after = np.gradient(smoothed_base_after)

                        # Histograma ajustado a formato similar a Mathematica
                        ax_left.hist(derivative_base_before, bins=25, color='black', alpha=0.5, label=f'Base Antes: {base_key}', edgecolor='black', density=True, log=True)
                        ax_left.hist(derivative_exp, bins=25, color=suffix_to_color[exp_suffix], alpha=0.7, label=f'Experimental: {exp_key}', edgecolor='black', density=True, log=True)
                        ax_left.set_title(f"{component} - Antes vs Exp", fontsize=30)
                        ax_left.set_xlabel("Valor Derivado", fontsize=25)
                        ax_left.set_ylabel("Probabilidad", fontsize=25)
                        ax_left.legend(fontsize=25)
                        ax_left.tick_params(axis='both', labelsize=35)

                        ax_right.hist(derivative_base_after, bins=25, color='black', alpha=0.5, label=f'Base Después: {base_key_after}', edgecolor='black', density=True, log=True)
                        ax_right.hist(derivative_exp, bins=25, color=suffix_to_color[exp_suffix], alpha=0.7, label=f'Experimental: {exp_key}', edgecolor='black', density=True, log=True)
                        ax_right.set_title(f"{component} - Después vs Exp", fontsize=30)
                        ax_right.set_xlabel("Valor Derivado", fontsize=25)
                        ax_right.set_ylabel("Probabilidad", fontsize=25)
                        ax_right.legend(fontsize=25)
                        ax_right.tick_params(axis='both', labelsize=35)

                    date, shift = title_info
                    shift_full = "MATUTINO" if shift == "M" else "VESPERTINO"
                    plt.suptitle(f"Comparaciones de Derivadas - {date} - {shift_full} - {lab}", fontsize=50)
                    plt.tight_layout(pad=5.0)
                    output_file = os.path.join(output_folder, f"deriv_{n:02d}_{exp_suffix.split('.')[0]}.pdf")
                    plt.savefig(output_file, bbox_inches='tight', format='pdf')
                    plt.close()

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def main():
    laboratorios = ["Lab_Betta", "Lab_Gamma"]
    turnos = ["M", "V"]

    for lab in laboratorios:
        for turno in turnos:
            experiment_folder, date_str = find_experiment_folder(root_folder, lab, turno)
            if not experiment_folder:
                print(f"No se encontró ningún directorio para {lab}, {turno}.")
                continue

            output_folder = os.path.join(experiment_folder, "Data Analysis", "Graphics", "Comparison_D[R]")
            os.makedirs(output_folder, exist_ok=True)

            title_info = (date_str, turno)

            data_per_file = analyze_folder(experiment_folder)
            plot_comparison_grid(data_per_file, output_folder, title_info, lab)

if __name__ == "__main__":
    main()


Analizando la carpeta raíz: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.M - Lab_Betta

Subdirectorio: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.M - Lab_Betta


Subdirectorio: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.M - Lab_Betta\Negra
   Archivo ignorado: 001mednegra.txt
   Archivo ignorado: 002mednegra.txt
   Encontrado y procesado: 01mednegra.txt
   Encontrado y procesado: 02mednegra.txt
   Encontrado y procesado: 03mednegra.txt
   Encontrado y procesado: 04mednegra.txt
   Encontrado y procesado: 05mednegra.txt


Subdirectorio: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.M - Lab_Betta\Roja
   Encontrado y procesado: 01medroja.txt
   Encontrado y procesado: 02medroja.txt
   Encontrado y procesado: 03medroja.txt
   Encontrado y procesado: 04medroja.txt


Subdirectorio: E:\Pruebas\14Sep24\14Sep24 - Lab_Betta\14Sep24.M - Lab_Betta\Roja\Pesajes Roja
   Archivo ignorado: 01medroja-pesajes.log
   Archivo ignorado: 02medroja-pesajes.log
   Archivo ignorado: 03medroja-pesaje


## 12

$$
    \Huge \text{Comparison Time Series Laser}
$$



In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re

def read_sensor_data(filepath):
    data = []
    num_elements = None
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    numbers = parts[1].strip('()').split(',')
                    try:
                        number_list = [float(num.strip()) for num in numbers]
                        if num_elements is None:
                            num_elements = len(number_list)
                        if len(number_list) == num_elements:
                            data.append(number_list)
                        else:
                            print(f"Inconsistent number of elements in file {filepath}, line: {line.strip()}")
                    except ValueError:
                        print(f"Value error in file {filepath}, line: {line.strip()}")
                        continue
    except UnicodeDecodeError:
        print(f"Unicode decode error in file {filepath}")
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
    return np.array(data).T

def analyze_folder(root_folder):
    data_per_file = {}
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d != "Data Analysis"]
        for filename in files:
            if filename.endswith(("negra.txt", "roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt")) and not filename.startswith("00"):
                file_path = os.path.join(root, filename)
                data = read_sensor_data(file_path)
                if data.size > 0:
                    data_per_file[filename] = data
    return data_per_file

def smooth_data(data, window_length=10, polyorder=3):
    if len(data) < window_length:
        window_length = len(data) - (len(data) % 2 == 0)
    return savgol_filter(data, window_length=window_length, polyorder=polyorder)

def plot_comparison_grid(data_per_file, output_folder, title_info, lab):
    suffix_to_color = {
        "negra.txt": "black",
        "roja.txt": "red",
        "morada.txt": "purple",
        "azul.txt": "blue",
        "verde.txt": "green",
        "amarilla.txt": "yellow",
        "naranja.txt": "orange"
    }
    component_names = ["FRONTAL", "TRASERO", "IZQUIERDA", "DERECHA", "SUPERIOR", "INFERIOR"]

    for base_key in sorted(data_per_file.keys()):
        if "negra.txt" in base_key:
            try:
                n = int(re.search(r'\d+', base_key.split('med')[0]).group())
            except (ValueError, AttributeError):
                print(f"El archivo {base_key} no contiene un número válido para conversión.")
                continue

            base_data_before = data_per_file[base_key]
            for exp_suffix in ["roja.txt", "morada.txt", "azul.txt", "verde.txt", "amarilla.txt", "naranja.txt"]:
                exp_key = f"{n:02d}med{exp_suffix}"
                base_key_after = f"{n+1:02d}mednegra.txt"
                if exp_key in data_per_file and base_key_after in data_per_file:
                    exp_data = data_per_file[exp_key]
                    base_data_after = data_per_file[base_key_after]
                    fig, axes = plt.subplots(6, 2, figsize=(40, 60), dpi=300)
                    axes = axes.reshape(-1, 2)
                    for i, component in enumerate(component_names):
                        ax_before = axes[i, 0]
                        ax_after = axes[i, 1]
                        smoothed_base_before = smooth_data(base_data_before[i])
                        smoothed_exp = smooth_data(exp_data[i])
                        smoothed_base_after = smooth_data(base_data_after[i])
                        ax_before.plot(smoothed_base_before, label=f'Línea Base: {base_key}', color='black', linewidth=2)
                        ax_before.plot(smoothed_exp, label=f'Experimental: {exp_key}', color=suffix_to_color[exp_suffix], linewidth=2)
                        ax_before.set_title(f"Componente {component} (Antes)", fontsize=25)
                        ax_before.set_xlabel("Registros", fontsize=25)
                        ax_before.set_ylabel("Volts", fontsize=25)
                        ax_before.tick_params(axis='both', which='major', labelsize=20)
                        ax_before.legend(fontsize=30)
                        ax_after.plot(smoothed_base_after, label=f'Línea Base: {base_key_after}', color='black', linewidth=2)
                        ax_after.plot(smoothed_exp, label=f'Experimental: {exp_key}', color=suffix_to_color[exp_suffix], linewidth=2)
                        ax_after.set_title(f"Componente {component} (Después)", fontsize=25)
                        ax_after.set_xlabel("Registros", fontsize=25)
                        ax_after.set_ylabel("Volts", fontsize=25)
                        ax_after.tick_params(axis='both', which='major', labelsize=20)
                        ax_after.legend(fontsize=30)
                    date, shift = title_info
                    shift_full = "MATUTINO" if shift == "M" else "VESPERTINO"
                    color_name = exp_suffix.split('.')[0].upper()
                    output_file = os.path.join(output_folder, f"comparacion_{n:02d}_{n+1:02d}_base_{n}_{exp_suffix.split('.')[0]}.pdf")
                    plt.suptitle(f"COMPARACIÓN BASE {n:02d} Y DESPUÉS {n+1:02d} DE INTERVENCIÓN - {date} - {shift_full} - INTERVENCIÓN: {color_name} - LABORATORIO: {lab}", fontsize=35)
                    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                    plt.savefig(output_file, bbox_inches='tight', format='pdf')
                    plt.close()

def find_experiment_folder(root_folder, lab, turno):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and turno in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def main():
    # Definir los laboratorios y turnos
    laboratorios = ["Lab_Betta", "Lab_Gamma"]
    turnos = ["M", "V"]


    for lab in laboratorios:
        for turno in turnos:
            # Encontrar la carpeta del experimento correspondiente al laboratorio y turno
            experiment_folder, date_str = find_experiment_folder(root_folder, lab, turno)
            if not experiment_folder:
                print(f"No se encontró ningún directorio para {lab}, {turno}.")
                continue

            # Acceder directamente a la carpeta del experimento para procesar datos sin entrar a "Data Analysis"
            output_folder = os.path.join(experiment_folder,"Data Analysis", "Graphics", "Comparison")
            os.makedirs(output_folder, exist_ok=True)  # Crear la carpeta de gráficos si no existe

            title_info = (date_str, turno)

            # Analizar los archivos en la carpeta del experimento directamente
            data_per_file = analyze_folder(experiment_folder)
            # Generar las gráficas y guardarlas en la carpeta de gráficos
            plot_comparison_grid(data_per_file, output_folder, title_info, lab)

if __name__ == "__main__":
    main()


No se encontró ningún directorio para Lab_Betta, V.
No se encontró ningún directorio para Lab_Gamma, V.
